# Agentic RAG (Retreieval Augmented Generation) System using Amazon Bedrock Knowledge Bases and LangGraph

This notebook implements a self-corrective RAG (Retrieval-Augmented Generation) workflow using strands agent for orchestration. The system uses several advanced techniques to improve answer accuracy:

1. **Query expansion** - Looks up domain terms to transform queries with added context
2. **Query decomposition** - Breaks down complex questions when needed
3. **Response quality** - Evaluates if responses quality is accceptable
4. **Self-correction** - Evaluates response quality and relevance, taking corrective action if needed

## Pre-requisites
1. Run notebook 1 (01-setup-create-insert-dynamodb.ipynb) to create and populate the DynamoDB table, with sample lookup data. 

## Import Stored Variables

Retrieve the DynamoDB table name from the previous notebook using Jupyter's `%store -r` magic command.

In [1]:
%store -r table_name

no stored variable or alias table_name


## Install Required Libraries

Install necessary packages for implementing the RAG system:
- strands for building AI agents
- boto3 for python AWS SDK
- Other utility libraries for handling asynchronous operations, logging, etc.


In [1]:
%%capture --no-stderr
%pip install --force-reinstall -r requirements.txt

NOTE: Make sure to manually restart the notebook after installing packages.

In [2]:
# # restart kernel
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")

## Configure Amazon Bedrock Services

Set up the Amazon Bedrock services:
1. Choose the foundation model (Claude 3 Sonnet in this case)
2. Configure the AWS region
3. Initialize Bedrock clients (runtime and agent-runtime)
4. Set the Knowledge Base ID for RAG retrieval

In [27]:
# Model configurations (used for query expansion)

#modelId = 'anthropic.claude-3-sonnet-20240229-v1:0'
modelId = 'us.anthropic.claude-haiku-4-5-20251001-v1:0'
print(f'Using modelId: {modelId}')

Using modelId: us.anthropic.claude-haiku-4-5-20251001-v1:0


In [28]:
! echo $AWS_PROFILE
! AWS_PROFILE=None
! echo 'NewProfile:'$AWS_PROFILE


NewProfile:


## Connecting to AWS Environment

In [29]:
import os
import pprint
import boto3
import json, sys
from datetime import datetime
import json
import time
import boto3
import asyncio
import logging
import inspect
from typing import List

# Strands Agents imports
from strands import Agent, tool

from utils.knowledge_base import BedrockKnowledgeBase

print(f"Old Profile:{os.environ.get('AWS_PROFILE')}")
os.environ.pop('AWS_PROFILE', None)
print(f"New Profile:{os.environ.get('AWS_PROFILE')}")


# Clear any cached credentials in boto3
#boto3.DEFAULT_SESSION = None

# Uncomment and set your AWS profile if using a named profile
# import os
#os.environ['AWS_PROFILE'] = 'heyjack+autotecture-Admin'
#os.environ['AWS_DEFAULT_REGION'] = 'us-west-2'

# Create a fresh session
session = boto3.Session()
region = session.region_name

print("=== AWS Connection Test ===")
try:
    # Test AWS connection first
    sts_client = session.client('sts')
    identity = sts_client.get_caller_identity()
    account_id = identity["Account"]
    print(f"✓ AWS connection successful!")
    print(f"Account: {account_id}")
    print(f"User/Role: {identity.get('Arn')}")
    print(f"Region: {region}")
    
    # Create AWS clients using the session
    s3_client = session.client('s3')
    bedrock = session.client(
        service_name = 'bedrock-runtime',
        region_name = region,
    )
    bedrock_agent_client = session.client('bedrock-agent')
    bedrock_agent_runtime_client = session.client('bedrock-agent-runtime')
    
    print("✓ All AWS clients created successfully!")
    
except Exception as e:
    print(f"✗ AWS connection failed: {e}")
    print("\n=== TROUBLESHOOTING ===")
    print("1. Restart Jupyter kernel: Kernel → Restart Kernel")
    print("2. Check AWS profile: aws configure list")
    print("3. Test in terminal: aws sts get-caller-identity")
    print("4. Set profile: os.environ['AWS_PROFILE'] = 'your-profile'")
    raise

logging.basicConfig(format='[%(asctime)s] p%(process)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)
region, account_id

Old Profile:None
New Profile:None
=== AWS Connection Test ===
✓ AWS connection successful!
Account: 211125637749
User/Role: arn:aws:sts::211125637749:assumed-role/AmazonSageMaker-ExecutionRole-20251107T111685/SageMaker
Region: us-west-2
✓ All AWS clients created successfully!


('us-west-2', '211125637749')

## Set up for creating a Knowledge Base

In [101]:
import time

# Get the current timestamp
current_time = time.time()

# Format the timestamp as a string
timestamp_str = time.strftime("%Y%m%d%H%M%S", time.localtime(current_time))[-7:]
# Create the suffix using the timestamp
suffix = f"{timestamp_str}"

knowledge_base_name = f"bedrock-kb-corrective-rag-{suffix}"
knowledge_base_description = "Octank financial data with s3 as data source."

bucket_name = f'{knowledge_base_name}-{account_id}'

data_bucket_name = f'bedrock-kb-{suffix}-1' # replace it with your first bucket name.

data_sources=[{"type": "S3", "bucket_name": data_bucket_name}]
                
pp = pprint.PrettyPrinter(indent=2)

### Creating Knowledge Base

In [ ]:
knowledge_base = BedrockKnowledgeBase(
    kb_name=f'{knowledge_base_name}',
    kb_description=knowledge_base_description,
    data_sources=data_sources,
    chunking_strategy = "FIXED_SIZE", 
    suffix = f'{suffix}-f'
)

#### Upload data to S3 Bucket data source

In [103]:
def upload_directory(path, bucket_name):
    for root, dirs, files in os.walk(path):
        for file in files:
            file_to_upload = os.path.join(root, file)
            if file not in ["LICENSE", "NOTICE", "README.md", "podcastdemo.mp3","bda.m4v"]:
                print(f"uploading file {file_to_upload} to {bucket_name}")
                s3_client.upload_file(file_to_upload, bucket_name, file)
            else:
                print(f"Skipping file {file_to_upload}")

upload_directory("./synthetic_dataset", data_bucket_name)

Skipping file ./synthetic_dataset/podcastdemo.mp3
Skipping file ./synthetic_dataset/LICENSE
Skipping file ./synthetic_dataset/NOTICE
Skipping file ./synthetic_dataset/README.md
Skipping file ./synthetic_dataset/bda.m4v
uploading file ./synthetic_dataset/octank_financial_10K.pdf to bedrock-kb-2150725-1


### Start ingestion job
Once the KB and data source(s) created, we can start the ingestion job for each data source.
During the ingestion job, KB will fetch the documents in the data source, pre-process it to extract text, chunk it based on the chunking size provided, create embeddings of each chunk and then write it to the vector database, in this case OSS.

NOTE: Currently, you can only kick-off one ingestion job at one time.

In [ ]:
# ensure that the kb is available
time.sleep(30)
# sync knowledge base
knowledge_base.start_ingestion_job()

In [ ]:
# keep the kb_id for invocation later in the invoke request
kb_id = knowledge_base.get_knowledge_base_id()
%store kb_id

### Setting KB ID Manually for the already created KB. Uncomment and run the below cell if you want to use the already created KB.

In [30]:
# Development Account
#kb_id = "KA1NFBFTX0"
#region = "us-east-1"

#Production Account
kb_id = "C4ENDDUOSQ"
region = "us-west-2"

table_name = "glossary-2"
#modelId = 'anthropic.claude-3-sonnet-20240229-v1:0'
modelId = 'us.anthropic.claude-haiku-4-5-20251001-v1:0'


## Basic RAG Flow

Let's test our RAG workflow with some basic queries now

In [23]:
def retrieve_and_generate(question: str) -> str:
    """Generate response using Bedrock RAG with optional decomposition."""
    print("---CALL RETRIEVE and GENERATE---")
    
    config = {
        "type": "KNOWLEDGE_BASE",
        "knowledgeBaseConfiguration": {
            'knowledgeBaseId': kb_id,
            "modelArn": f"arn:aws:bedrock:{region}:{account_id}:inference-profile/{modelId}",
            "retrievalConfiguration": {
                "vectorSearchConfiguration": {"numberOfResults": 5}
            }
        }
    }
        
    response = bedrock_agent_runtime_client.retrieve_and_generate(
        input={"text": question},
        retrieveAndGenerateConfiguration=config
    )
    return response['output']['text']

"""Main function to test the Basic RAG Flow."""
print("\n" + "="*80)
print("🚀 BASIC RAG FLOW - TESTING")
print("="*80)

# Test Basic questions
test_questions = [
    "What is the fair value of HTM portfolio?",
    "What are the main business segments of Octank Financial?", 
    "How many employees does Octank Financial have?"
]

# Test Complex Question
test_questions = [
# "What is octank tower and how does the whistleblower scandal hurt the company and its image?",
# "Provide list of DCMs.",
# "Provide list of DCMs and how many regional offices?"]

# Process each question
for i, question in enumerate(test_questions, 1):
    print(f"\n📋 Question {i}: {question}")
    print("-" * 60)
    
    try:
        # Generate RAG response
        start_time = time.time()
        answer = retrieve_and_generate(question)
        end_time = time.time()
        
        print(f"✅ Response generated in {end_time - start_time:.2f} seconds")
        print(f"\n🤖 Answer:")
        print(answer)
        
    except Exception as e:
        print(f"❌ Error processing question: {e}")
    
    print("\n" + "="*80)



🚀 BASIC RAG FLOW - TESTING

📋 Question 1: What is octank tower and how does the whistleblower scandal hurt the company and its image?
------------------------------------------------------------
---CALL RETRIEVE and GENERATE---
✅ Response generated in 4.58 seconds

🤖 Answer:
I could not find any information about "Octank Tower" in the provided search results. The search results do not contain details about what Octank Tower is or whether it exists. The whistleblower scandal significantly hurt Octank Financial in multiple ways. Person X, the company's Chief Financial Officer, was accused of insider trading and other illegal activities. Following the investigation announcement, the company's stock price plummeted, causing substantial losses for shareholders.

Beyond financial impacts, the scandal damaged the company's reputation, with many questioning its ability to maintain ethical standards and protect investors. Employee morale suffered considerably, with staff feeling demoralized an

## Improving RAG Accuracy

### Query Decomposition
Question: What is octank tower and how does the whistleblower scandal hurt the company and its image?

In [8]:
def decompose_and_generate(question: str) -> str:
    """Generate response using Bedrock RAG with optional decomposition."""
    print("---CALL DECOMPOSE and GENERATE---")
    
    config = {
        "type": "KNOWLEDGE_BASE",
        "knowledgeBaseConfiguration": {
            'knowledgeBaseId': kb_id,
            "modelArn": f"arn:aws:bedrock:{region}:{account_id}:inference-profile/{modelId}",
            "retrievalConfiguration": {
                "vectorSearchConfiguration": {"numberOfResults": 5}
            }
        }
    }

    config["knowledgeBaseConfiguration"]["orchestrationConfiguration"] = {
    'queryTransformationConfiguration': {'type': 'QUERY_DECOMPOSITION'}
    }

    response = bedrock_agent_runtime_client.retrieve_and_generate(
        input={"text": question},
        retrieveAndGenerateConfiguration=config
    )
    return response['output']['text']


"""Main function to test the Query Decomposition RAG Flow."""
print("\n" + "="*80)
print("🚀 Query Decomposition")
print("="*80)

# Test Complex Question
test_questions = ["What is octank tower and how does the whistleblower scandal hurt the company and its image?"]


# Process each question
for i, question in enumerate(test_questions, 1):
    print(f"\n📋 Question {i}: {question}")
    print("-" * 60)
    
    try:
        # Generate RAG response
        start_time = time.time()
        answer = decompose_and_generate(question)
        end_time = time.time()
        
        print(f"✅ Response generated in {end_time - start_time:.2f} seconds")
        print(f"\n🤖 Answer:")
        print(answer)
        
    except Exception as e:
        print(f"❌ Error processing question: {e}")
    
    print("\n" + "="*80)



🚀 Query Decomposition

📋 Question 1: What is octank tower and how does the whistleblower scandal hurt the company and its image?
------------------------------------------------------------
---CALL DECOMPOSE and GENERATE---
✅ Response generated in 4.97 seconds

🤖 Answer:
Octank Tower is an iconic office building designed by renowned architect PersonA that symbolizes the company's commitment to innovation and sustainability. The structure contains 800,000 square feet of office space and houses more than 3,500 employees across various departments including research and development, marketing, finance, and human resources. The building features cutting-edge technology, including a sophisticated building management system that optimizes energy efficiency and indoor air quality. It has received several awards for its green initiatives, including LEED Platinum certification, and includes eco-friendly amenities such as solar panels, rainwater harvesting systems, and electric vehicle charging

### Query Expansion

Question: Provide list of DCMs.

Note that we have created a query expansion agent which is using lookup_term tool to search for the domain related term from the pre defined registry in DynamoDB.

The query expansion agent is then used to expand the query with the domain terms from the glossary.

In [9]:
# Strands Agents imports
from strands import Agent, tool

@tool
def lookup_term(term):
    
    dynamodb = session.resource('dynamodb')
    table = dynamodb.Table(table_name)

    try:
        response = table.get_item(
            Key={
                'term': term.lower(),
            },
            ProjectionExpression='term_definition'
        )
    except Exception as e:
        print(f"Error looking up term '{term}': {e}")
        return None

    if 'Item' in response:
        definition = response['Item'].get('term_definition')
        print(f'Tool result: {definition}')
        return definition
    else:
        print(f"Term '{term}' not found in the table.")
        return " "


def query_expansion(question: str, terms: str) -> str:
    """
    Expand query using term definitions via intelligent agent-based lookup.
    
    Args:
        question: The original user question
        terms: Comma-separated list of terms to look up
        
    Returns:
        Transformed/expanded question with meaningful information
    """
    print(f"---QUERY EXPANSION---")
    print(f"Original question: {question}")
    print(f"Terms to expand: {terms}")
    
    # Create a specialized query expansion agent
    expansion_agent = Agent(
        name="query_expansion_agent",
        model=modelId,
        tools=[lookup_term],
        system_prompt=f"""You're provided a tool named 'lookup_term' to transform or reformulate the user question.
You can call the tool multiple times in the same response if required.

'lookup_term' can be used to return the definition of terms from the glossary.

Available terms to look up: [{terms}]

Instructions:
1. Call the tool 'lookup_term' for each relevant term from the provided list
2. Only use the return values of this tool to get the definition of the term
3. Use the term definitions and the question to transform the question with meaningful information
4. DO NOT change the meaning of the original question
5. If no value is returned by tool, return the original question as-is
6. Make sure the final output is the single transformed question only
7. Don't make reference to the tools in your final answer

Output format:
- DO NOT add any preamble to the output such as "The transformed question is:"
- Output should only include the transformed question
- Don't make reference to the tools in your final answer
- DO NOT answer the question, only provide transformed question as final output"""
    )
    
    try:
        # Create the expansion prompt
        expansion_prompt = f"""Transform this question using relevant term definitions: {question}

Available terms to look up: {terms}

Remember: Only output the transformed question, no preamble or explanations."""
        
        # Get the expanded query from the agent
        result = expansion_agent(expansion_prompt)
        transformed_query = str(result).strip()
        
        print(f"Transformed query: {transformed_query}")
        return transformed_query
        
    except Exception as e:
        print(f"Error in query expansion: {e}")
        print(f"Returning original question: {question}")
        return question


"""Main function to test the Basic RAG Flow."""
print("\n" + "="*80)
print("🚀 Query Expansion Flow")
print("="*80)

# Test question containing some unique abbreviation
test_questions = ["Provide list of DCMs."]

# Process each question
for i, question in enumerate(test_questions, 1):
    print(f"\n📋 Question {i}: {question}")
    print("-" * 60)
    
    try:
        # Generate RAG response
        start_time = time.time()
        expanded_query = query_expansion(question, "DCM")
        answer = retrieve_and_generate(expanded_query)
        end_time = time.time()
        
        print(f"✅ Response generated in {end_time - start_time:.2f} seconds")
        print(f"\n🤖 Answer:")
        print(answer)
        
    except Exception as e:
        print(f"❌ Error processing question: {e}")
    
    print("\n" + "="*80)

[2025-11-28 18:41:07,918] p992 {metrics.py:478} INFO - Creating Strands MetricsClient



🚀 Query Expansion Flow

📋 Question 1: Provide list of DCMs.
------------------------------------------------------------
---QUERY EXPANSION---
Original question: Provide list of DCMs.
Terms to expand: DCM

Tool #1: lookup_term
Tool result: Disclosure Committee Members
Provide list of Disclosure Committee Members.Transformed query: Provide list of Disclosure Committee Members.
---CALL RETRIEVE and GENERATE---
✅ Response generated in 4.20 seconds

🤖 Answer:
The Disclosure Committee Members at Octank Financial are:

1. PersonA - Chief Financial Officer
2. PersonB - General Counsel
3. PersonC - Vice President of Investor Relations

The disclosure committee is comprised of senior executives from various departments, including finance, legal, and investor relations, and meets regularly to review disclosure practices and ensure compliance with applicable laws and regulations.



### Self Corrective RAG Agent

**How do we design a system that can use appropriate technique(s) with a goal of improving answer accuracy?**

This is where concept of Self Corrective Recursive Agentic RAG 🔄 comes into picture.
- 📄 **First retrieve raw documents** to assess context relevance
- 🎯 **Select initial strategy** based on document relevance and question complexity
- 🔍 **Generate response and check quality** with AI agent
- 🤔 **If quality < threshold**, re-select strategy based on quality feedback
- 🧠 **Central multi agent adapts strategy** (query decomposition, query expansion, or combination) to improve accuracy

🎯 Define Thresholds
- Quality Metrics
- Quality threshold
- Attempts threshold

#### Multi-Agent Architecture Overview

**System Flow:**

📝 **User Question** → 🧠 **Central Agent** → 📄 **Retrieve Documents** → 🎯 **Select Strategy** → ⚡ **Generate Response** → 🔍 **Quality Check**

**Decision Points:**
- Quality ≥ 9.0 → ✅ **DONE**
- Quality < 9.0 → 🔄 **IMPROVE** 
- Attempts ≥ 3 → 📋 **BEST**

**Improvement Strategies:**

| Strategy | When Used | Process |
|----------|-----------|----------|
|**Query Expansion** | Acronyms/Terms | expansion_tool → Sub-Agent → DynamoDB → Enhanced RAG |
|**Query Decomposition** | Complex Questions | decompose_rag → Bedrock Runtime → Enhanced Response |
|**Combined** | Combination | expansion_tool + decompose_rag → Enhanced Response |

**🎯 Key Agents:**
- 🧠 **Central Agent**: Multi Agent orchestrator with 5 tools
- 🤖 **Query Expansion Sub-Agent**: Handles acronym expansion via DynamoDB
- 🔍 **Quality Evaluation Sub-Agent**: Scores responses (0-10 scale)

**🔧 Tools:**
1. **retrieve_and_generate_tool**: Basic RAG
2. **decompose_and_generate_tool**: Query decomposition
3. **query_expansion_agent_tool**: Query expansion
4. **retrieve_documents_tool**: Retrieve raw documents 
5. **evaluate_response_quality_with_agent**: Quality assessment

## Create Tools and Sub Agents

In [31]:
# Self-Corrective RAG Configuration
MAX_RETRY_ATTEMPTS = 3
QUALITY_THRESHOLD = 9.0  # Out of 10
qualityCheckModelId = 'us.anthropic.claude-sonnet-4-5-20250929-v1:0'

@tool
def retrieve_documents_tool(question: str) -> str:
    """Retrieve relevant documents from Knowledge Base without generation.
    
    Use this tool FIRST to assess document relevancy before deciding 
    on the appropriate generation strategy.
    
    Args:
        question: The user's question to retrieve relevant documents for
        
    Returns:
        Formatted string with retrieved document chunks
    """
    print("---TOOL: RETRIEVE DOCUMENTS---")
    print(f"Question: {question}")
    
    try:
        response = bedrock_agent_runtime_client.retrieve(
            knowledgeBaseId=kb_id,
            retrievalQuery={'text': question}
        )
        
        contexts = []
        for context in response["retrievalResults"]:
            contexts.append(context['content']['text'])
        
        print(f"Retrieved {len(contexts)} document chunks")
        
        # Format as readable string for LLM
        result = f"Retrieved {len(contexts)} relevant documents:\n\n"
        for i, doc in enumerate(contexts, 1):
            result += f"Document {i}:\n{doc}\n\n---\n\n"
        
        return result
        
    except Exception as e:
        print(f"Error in retrieve_documents: {e}")
        return f"Error retrieving documents: {e}"

@tool
def retrieve_and_generate_tool(question: str) -> str:
    """Generate response using basic Bedrock RAG."""
    print("---TOOL: RETRIEVE and GENERATE---")
    print(f"Question: {question}")
    
    config = {
        "type": "KNOWLEDGE_BASE",
        "knowledgeBaseConfiguration": {
            'knowledgeBaseId': kb_id,
            "modelArn": f"arn:aws:bedrock:{region}:{account_id}:inference-profile/{modelId}",
            "retrievalConfiguration": {
                "vectorSearchConfiguration": {"numberOfResults": 5}
            }
        }
    }
        
    try:
        response = bedrock_agent_runtime_client.retrieve_and_generate(
            input={"text": question},
            retrieveAndGenerateConfiguration=config
        )
        response_text = response['output']['text']
        print(f"Response: {response_text}")
        print(f"Response length: {len(response_text)} characters")
        return response_text
    except Exception as e:
        print(f"Error in retrieve_and_generate: {e}")
        return f"Error generating response: {e}"

@tool
def decompose_and_generate_tool(question: str) -> str:
    """Generate response using Bedrock RAG with query decomposition."""
    print("---TOOL: DECOMPOSE and GENERATE---")
    print(f"Question: {question}")
    
    config = {
        "type": "KNOWLEDGE_BASE",
        "knowledgeBaseConfiguration": {
            'knowledgeBaseId': kb_id,
            "modelArn": f"arn:aws:bedrock:{region}:{account_id}:inference-profile/{modelId}",
            "retrievalConfiguration": {
                "vectorSearchConfiguration": {"numberOfResults": 5}
            }
        }
    }

    config["knowledgeBaseConfiguration"]["orchestrationConfiguration"] = {
        'queryTransformationConfiguration': {'type': 'QUERY_DECOMPOSITION'}
    }

    try:
        response = bedrock_agent_runtime_client.retrieve_and_generate(
            input={"text": question},
            retrieveAndGenerateConfiguration=config
        )
        response_text = response['output']['text']
        print(f"Response: {response_text}")
        print(f"Response length: {len(response_text)} characters")
        return response_text
    except Exception as e:
        print(f"Error in decompose_and_generate: {e}")
        return f"Error generating response with decomposition: {e}"

@tool
def lookup_term(term: str) -> str:
    """Look up term definition from DynamoDB glossary table."""
    dynamodb = session.resource('dynamodb')
    table = dynamodb.Table(table_name)

    try:
        response = table.get_item(
            Key={'term': term.lower()},
            ProjectionExpression='term_definition'
        )
    except Exception as e:
        print(f"Error looking up term '{term}': {e}")
        return f"Error looking up term: {e}"

    if 'Item' in response:
        definition = response['Item'].get('term_definition')
        print(f'Tool result for {term}: {definition}')
        return definition
    else:
        print(f"Term '{term}' not found in the table.")
        return f"Term '{term}' not found in glossary"

@tool
def query_expansion_agent_tool(question: str) -> str:
    """
    Expand query using term definitions via intelligent sub-agent with lookup capabilities.
    This creates a specialized query expansion agent that automatically identifies and expands domain acronym terms.
    
    Args:
        question: The original user question
        
    Returns:
        Transformed/expanded question with meaningful information
    """
    print(f"---TOOL: QUERY EXPANSION AGENT---")
    print(f"Original question: {question}")
    
    # Create a specialized query expansion agent (same as QueryExpansionFlow.py)
    expansion_agent = Agent(
        name="query_expansion_agent",
        model=modelId,
        tools=[lookup_term],
        system_prompt="""You're provided a tool named 'lookup_term' to transform or reformulate the user question.
You can call the tool multiple times in the same response if required.

'lookup_term' can be used to return the definition of terms from the glossary.

Instructions:
1. Analyze the question to identify  abbreviations, or acronyms that might need expansion
2. Call the tool 'lookup_term' for each relevant term you identify
3. ONLY use the return values of this tool to get the definition of the term
4. Use the term definitions and the question to transform the question with meaningful information
5. DO NOT change the meaning of the original question
6. If NO value is returned by tool, return the original question AS-IS WITHOUT ANY MODIFICATION
7. Make sure the final output is the single transformed question only
8. Don't make reference to the tools in your final answer


Output format:
- DO NOT add any preamble to the output such as "The transformed question is:"
- Output should only include the transformed question
- Don't make reference to the tools in your final answer
- DO NOT answer the question, only provide transformed question as final output

"""
    )
    
    try:
        # Create the expansion prompt
        expansion_prompt = f"""Transform this question using relevant term definitions: {question}

Analyze the question for abbreviation or acronyms terms and ONLY use the lookup_term tool to get their definitions.
Remember: Only output the transformed question, no preamble or explanations."""
        
        # Get the expanded query from the agent
        result = expansion_agent(expansion_prompt)
        transformed_query = str(result).strip()
        
        print(f"Transformed query: {transformed_query}")
        return transformed_query
        
    except Exception as e:
        print(f"Error in query expansion: {e}")
        print(f"Returning original question: {question}")
        return question

@tool
def evaluate_response_quality_with_agent(question: str, response: str) -> str:
    """Evaluate the quality of a RAG response using an intelligent agent. LLM as a judge"""
    print(f"---TOOL: EVALUATE RESPONSE QUALITY WITH AGENT---")
    print(f"Question: {question}")
    print(f"Response length: {len(response)} characters")
    
    # Create a specialized quality evaluation agent
    quality_evaluator = Agent(
        name="quality_evaluator_agent",
        model=qualityCheckModelId,
        system_prompt=f"""You are an expert response quality evaluator. Your job is to assess RAG responses based on three key criteria and provide actionable recommendations.

## Evaluation Criteria (0-10 scale):
1. **Relevance**: How well does the response address the original question?
2. **Completeness**: Does the response provide comprehensive information? Is it not able to find some information?
3. **Factual Accuracy**: Is the information presented accurate and consistent?

## Evaluation Guidelines:
- Score each criterion from 0-10 (10 being excellent)
- Calculate overall quality as the average of the three scores
- Be thorough but fair in your evaluation
- Consider the context and complexity of the question

## Response Format (MUST follow exactly):
```
QUALITY EVALUATION:
Relevance: X/10 - [detailed explanation of why this score]
Completeness: X/10 - [detailed explanation of why this score]
Factual Accuracy: X/10 - [detailed explanation of why this score]
Overall Quality: X.X/10
Status: [EXCELLENT/GOOD/FAIR/POOR]
```
Provide detailed explanations for each score to help improve the system
"""
    )
    
    try:
        # Create the evaluation prompt
        evaluation_prompt = f"""Please evaluate this RAG response:

ORIGINAL QUESTION: {question}

RAG RESPONSE: {response}

Please provide a thorough quality evaluation following the specified format."""
        
        # Get the quality evaluation from the agent
        evaluation_result = quality_evaluator(evaluation_prompt)
        evaluation_text = str(evaluation_result).strip()
        
        print(f"Agent-based quality evaluation completed")
        #print(f"Evaluation result: {evaluation_text}")
        
        return evaluation_text
        
    except Exception as e:
        print(f"Error in agent-based quality evaluation: {e}")
        # Fallback to a simple evaluation
        fallback_result = f"""QUALITY EVALUATION:
Relevance: 5/10 - Error in evaluation, using fallback
Completeness: 5/10 - Error in evaluation, using fallback  
Factual Accuracy: 5/10 - Error in evaluation, using fallback
Overall Quality: 5.0/10
Status: FAIR
Error: {e}"""
        return fallback_result

## Create the Central Agent

Create a single Strands Agent that orchestrates the entire self-corrective RAG workflow using the tools defined above. The agent will intelligently decide when to use query expansion, decomposition, or both based on the question and intermediate results.

In [32]:
def create_central_self_correction_agent() -> Agent:
    """Create the central self-correction agent with all available tools."""
    
    all_tools = [
        retrieve_documents_tool,
        retrieve_and_generate_tool,
        decompose_and_generate_tool,
        query_expansion_agent_tool,
        evaluate_response_quality_with_agent
    ]
    
    return Agent(
        name="central_self_correction_agent",
        model=qualityCheckModelId,
        tools=all_tools,
        system_prompt=f"""You are a Central Self-Correction Agent responsible for providing the best possible answers to user questions using a flexible, adaptive RAG workflow.

## Your Strategy:
1. **ALWAYS Start with Document Retrieval**: Use 'retrieve_documents_tool' FIRST to assess context relevancy. This tool should be used ONLY ONCE at the beginning.
2. **Evaluate Retrieved Context**: Based on the retrieved documents, determine if they contain relevant information
3. **Select Generation Strategy**: 
   - If retrieved documents context is highly relevant to the user question: Use 'retrieve_and_generate_tool'
   - If retrieved documents context lacks relevance or are partially relevant to the user question and user question is a multipart question containing multiple sub questions: DIRECTLY proceed with 'decompose_and_generate_tool'
   - If retrieved documents context lacks relevance and user question contains acronyms: Use 'query_expansion_agent_tool'. As a next step use decompose_and_generate for multi part user questions. If it's not a multi part question use retrieve_and_generate_tool.
4. **Evaluate Quality**: Use 'evaluate_response_quality_with_agent' to assess response
5. **Adapt Strategy**: If quality < {QUALITY_THRESHOLD}/10, try appropriate tool as suggested above to further refine the quality
6. **Limit Retries**: Maximum {MAX_RETRY_ATTEMPTS} attempts. If quality still not acceptable after max retries, return the best available response


## Available Tools:
- **retrieve_documents_tool**: Retrieve raw document chunks to assess relevancy (USE THIS FIRST)
- **retrieve_and_generate_tool**: Basic RAG retrieval and generation. DONOT USE this tool to generate the response if the user question is a multi-part quesion.
- **decompose_and_generate_tool**: Always Use this tool to generate the response if the user question is a multi-part question containing sub-queries and the returned context doesn't look complete for answering the question. This tool will decompose the user question into multiple queries, it will get the response in parallel for each query and combine and provide the final answer
- **query_expansion_agent_tool**: Intelligent sub-agent that automatically identifies and expands acronym terms
- **evaluate_response_quality_with_agent**: Intelligent agent-based response quality assessment
- Multiple tool may need to be used one after another to get to a good quality response


## Response Format:
Always provide a structured response with:
- **Strategy Used**: What approach you took (Example - Basic RAG -> Expansion -> Decomposition, etc.)
- **Quality Score**: The final quality score achieved
- **Attempts Made**: How many attempts were needed
- **Final Answer**: The best response you could generate

## Important Notes:
- Be adaptive and flexible in your approach
- Don't over-engineer simple questions (i.e. not a multi-part question) that work well with basic RAG
- Always remember to use decompose_and_generate_tool for multi part user questions containing sub queries within 
- Always explain your reasoning and strategy choices
- If multiple attempts fail, return the best available response with explanation
"""
    )

## Kickoff the Self Corrective RAG Workflow

In [33]:
# ============================================================================
# EXECUTION WORKFLOW - Central Agent with Flexible Tool Usage
# ============================================================================
import re

class SelfCorrectiveRAGResult:
    """Container for self-corrective RAG results with quality metrics."""
    
    def __init__(self):
        self.original_question = None
        self.final_answer = None
        self.quality_score = None
        self.strategies_used = []
        self.attempts_made = 0
        self.success = False
        self.execution_time = 0
        self.detailed_log = []

def execute_self_corrective_rag(question: str) -> SelfCorrectiveRAGResult:
    """Execute the self-corrective RAG workflow using the central agent."""
    
    print(f"\n{'='*80}")
    print(f"🧠 CENTRAL SELF-CORRECTIVE AGENTIC RAG WORKFLOW")
    print(f"{'='*80}")
    print(f"📋 Original Question: {question}")
    print(f"🎯 Quality Threshold: GOOD")
    print(f"🔄 Max Retry Attempts: {MAX_RETRY_ATTEMPTS}")
    print(f"{'='*80}")
    
    # Create the central agent
    central_agent = create_central_self_correction_agent()
    
    start_time = time.time()
    
    # Create enhanced prompt for the central agent
    enhanced_prompt = f"""
QUESTION: {question}

INSTRUCTIONS:
Please provide the best possible answer to this question
"""
    
    try:
        # Execute the central agent
        result_message = central_agent(enhanced_prompt)
        end_time = time.time()
        
        # Create result container
        result = SelfCorrectiveRAGResult()
        result.original_question = question
        result.final_answer = str(result_message).strip()
        result.execution_time = end_time - start_time
        result.success = True  # We'll let the agent determine success
        
        # Try to extract structured information from the response
        response_text = str(result_message)
        
        # Look for quality score in the response (multiple possible formats)
        quality_patterns = [
            r'Quality Score[:\s]*(\d+\.?\d*)',
            r'Overall Quality[:\s]*(\d+\.?\d*)',
            r'quality[:\s]*(\d+\.?\d*)/10'
        ]
        
        for pattern in quality_patterns:
            quality_match = re.search(pattern, response_text, re.IGNORECASE)
            if quality_match:
                result.quality_score = float(quality_match.group(1))
                result.success = result.quality_score >= QUALITY_THRESHOLD
                break
        
        # Look for strategy information
        if "Basic RAG" in response_text:
            result.strategies_used.append("Basic RAG")
        if "Expansion" in response_text or "expand" in response_text.lower():
            result.strategies_used.append("Query Expansion")
        if "Decomposition" in response_text or "decompose" in response_text.lower():
            result.strategies_used.append("Query Decomposition")
        
        # Count attempts mentioned
        attempts_match = re.search(r'(\d+)\s*attempt', response_text, re.IGNORECASE)
        if attempts_match:
            result.attempts_made = int(attempts_match.group(1))
        else:
            result.attempts_made = 1  # Default to 1 attempt
                
        return result
        
    except Exception as e:
        end_time = time.time()
        print(f"\n❌ ERROR in self-corrective RAG: {e}")
        
        # Create error result
        result = SelfCorrectiveRAGResult()
        result.original_question = question
        result.final_answer = f"Error processing question: {e}"
        result.success = False
        result.execution_time = end_time - start_time
        
        return result

"""Main function to test the Central Self-Corrective Agentic RAG workflow."""
print("\n" + "="*100)
print("🚀 CENTRAL SELF-CORRECTIVE AGENTIC RAG - TESTING WITH FLEXIBLE TOOL USAGE")
print("="*100)

# Test questions with varying complexity
test_questions = [
    "What is the fair value of HTM portfolio?",  # Simple with abbreviation - should try expansion
    "What are the main business segments of Octank Financial?",  # Moderate complexity
    "Provide list of DCMs.",  # Simple with domain acronym term - should try expansion
    "What is octank tower and how does the whistleblower scandal hurt the company and its image?",  # Complex multi-part - should try decomposition
    "Provide list of DCMs and how many regional offices?"  # Complex multi-part - should try decomposition
]

results = []

# Process each question
for i, question in enumerate(test_questions, 1):
    print(f"\n{'🔸' * 50}")
    print(f"TEST CASE {i}/{len(test_questions)}")
    print(f"{'🔸' * 50}")
    
    try:
        result = execute_self_corrective_rag(question)
        #results.append(result)
        
    except Exception as e:
        print(f"❌ Unexpected error in test case {i}: {e}")
        results.append(None)
    
    print(f"\n{'='*100}")




🚀 CENTRAL SELF-CORRECTIVE AGENTIC RAG - TESTING WITH FLEXIBLE TOOL USAGE

🔸🔸🔸🔸🔸🔸🔸🔸🔸🔸🔸🔸🔸🔸🔸🔸🔸🔸🔸🔸🔸🔸🔸🔸🔸🔸🔸🔸🔸🔸🔸🔸🔸🔸🔸🔸🔸🔸🔸🔸🔸🔸🔸🔸🔸🔸🔸🔸🔸🔸
TEST CASE 1/5
🔸🔸🔸🔸🔸🔸🔸🔸🔸🔸🔸🔸🔸🔸🔸🔸🔸🔸🔸🔸🔸🔸🔸🔸🔸🔸🔸🔸🔸🔸🔸🔸🔸🔸🔸🔸🔸🔸🔸🔸🔸🔸🔸🔸🔸🔸🔸🔸🔸🔸

🧠 CENTRAL SELF-CORRECTIVE AGENTIC RAG WORKFLOW
📋 Original Question: What is the fair value of HTM portfolio?
🎯 Quality Threshold: GOOD
🔄 Max Retry Attempts: 3
I'll help you find the best possible answer to your question about the fair value of the HTM (Held-to-Maturity) portfolio. Let me start by retrieving relevant documents to assess the context.
Tool #1: retrieve_documents_tool
---TOOL: RETRIEVE DOCUMENTS---
Question: What is the fair value of HTM portfolio?
Retrieved 5 document chunks
Excellent! The retrieved documents contain highly relevant information that directly answers the question. The context clearly states the fair value of the HTM portfolio. This is a straightforward question with clear information available, so I'll use the basic RAG approach.
Tool #2: retrieve_and_generate_tool
--

## End
